# BLER Table Sigmoid Fitting

This notebook fits the BLER, as a function of SNR, with a sigmoid and stores its center and
scale parameters, for every MCS table, MCS index, and code block size.

This allows for:
- Quick ACK/NACK generation in simulations
- Handy closed-form SINR estimation in SALAD algorithm

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    if gpu_num!="":
        print(f'\nUsing GPU {gpu_num}\n')
    else:
        print('\nUsing CPU\n')
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"

import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sionna.sys import PHYAbstraction
from sionna.phy.nr.utils import decode_mcs_index


Using GPU 0



E0000 00:00:1758879263.934888  230405 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758879263.938374  230405 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Utility functions

In [ ]:
def sigmoid(x, center=0.0, scale=1.0):
    """ Sigmoid function """
    return 1 / (1 + np.exp(-(np.array(x) - center) / scale))

def sigmoid_derivatives(x, center=0.0, scale=1.0):
    """
    Compute derivatives of sigmoid function with respect to center and scale parameters.
    
    Parameters
    ----------
    x : tf.Tensor
        Input tensor
    center : float
        Center point of the sigmoid (default: 0.0)
    scale : float
        Scale factor controlling the steepness (default: 1.0)
        
    Returns
    -------
    tuple of tf.Tensor
        Derivatives with respect to center and scale
    """
    x = np.array(x)

    # Compute sigmoid
    s = sigmoid(x, center=center, scale=scale)
    
    # Derivative with respect to x
    d_x = s * (1 - s) / scale

    # Derivative with respect to center
    d_center = - s * (1 - s) / scale
    
    # Derivative with respect to scale
    d_scale = - s * (1 - s) * (x - center) / scale**2
    
    return d_x, d_center, d_scale

# Define the loss function
def loss_gradient(snr_vec, bler_vec, center, scale):
    err = sigmoid(snr_vec, center=center, scale=scale) - 1 + np.array(bler_vec)

    _, d_center, d_scale = sigmoid_derivatives(snr_vec, center=center, scale=scale)
    
    d_center = np.sum(err * d_center)
    
    d_scale = np.sum(err * d_scale)
    
    return d_center, d_scale, np.sum(err**2)


def optimize_center_scale(snr_vec,
                          bler_vec,
                          max_n_iter=10000,
                          learning_rate=.1,
                          loss_thrs=.001):
    """
    Optimize center and scale parameters of the sigmoid function via gradient descent.
    """
    # Initial guess for center
    center = snr_vec[np.argmin(abs(bler_vec - .5))]

    # Initial guess for scale
    snr_val = snr_vec[np.argmin(abs(1 - bler_vec - 1/(1+np.exp(-2))))] 
    scale = (snr_val - center) / 2
    scale = max(scale, .1)

    loss_vec = []
    for i in range(max_n_iter):
        d_center, d_scale, loss = loss_gradient(snr_vec, bler_vec, center, scale)
        center = center - d_center * learning_rate
        scale = scale - d_scale * learning_rate
        loss_vec.append(loss)
        if loss < loss_thrs:
            break
    return center, scale, loss_vec


## Sigmoid fitting

Fit the SINR-vs-BLER tables with sigmoids with appropriate center and scale parameters

In [ ]:
# Max n. iterations for the fitting via gradient descent
max_n_iter = 100000
# Gradient descent learning rate
learning_rate = 0.1
# Convergence threshold
loss_thrs = .0005

# Instantiate PHY abstraction
phy_abs = PHYAbstraction()

cat2name = {0: 'PUSCH', 1: 'PDSCH'}

cat_vec = []
mcs_table_index_vec = []
mcs_vec = []
cbs_vec = []
center_vec = []
scale_vec = []
loss_vec = []
se_vec = []

# Iterate across all MCS categories and tables, MCS indices and code block sizes
for cat in phy_abs.bler_table['category'].keys():
    for mcs_table_index in [1, 2]:
        table = phy_abs.bler_table['category'][cat]['index'][mcs_table_index]['MCS']
        for mcs in table.keys():
            snr_vec = table[mcs]['SNR_db']
            for cbs in table[mcs]['CBS'].keys():
                bler_vec = np.array(table[mcs]['CBS'][cbs]['BLER'])
                # Fit SINR-vs-BLER table with sigmoid function
                center, scale, loss = optimize_center_scale(snr_vec,
                                                            bler_vec,
                                                            max_n_iter=max_n_iter,
                                                            learning_rate=learning_rate,
                                                            loss_thrs=loss_thrs)
                # Spectral efficiency
                mod_order, rate = decode_mcs_index(mcs,
                     table_index=mcs_table_index,
                     is_pusch=False)
                
                se_vec.append(rate.numpy() * mod_order.numpy())
                cat_vec.append(cat2name[cat])
                mcs_table_index_vec.append(mcs_table_index)
                mcs_vec.append(mcs)
                cbs_vec.append(cbs)
                center_vec.append(center)
                scale_vec.append(scale)
                loss_vec.append(loss[-1])


# Save results to csv
df = pd.DataFrame({'category': cat_vec,
                   'table_index': mcs_table_index_vec,
                   'MCS': mcs_vec,
                   'spectral_efficiency': se_vec,
                   'CBS_num_info_bits': cbs_vec,
                   'sigmoid_center_db': center_vec,
                   'sigmoid_scale_db': scale_vec,
                   'fit_loss': loss_vec})

Visualize the obtained data frame

In [ ]:
df.head(20)

,category,table_index,MCS,spectral_efficiency,CBS_num_info_bits,sigmoid_center_db,sigmoid_scale_db,fit_loss
0,PUSCH,1,3,0.490234,24,-2.879159,0.865388,0.000495
1,PUSCH,1,3,0.490234,100,-2.966159,0.500463,0.000500
2,PUSCH,1,3,0.490234,500,-2.841029,0.371086,0.000499
3,PUSCH,1,3,0.490234,1000,-2.685409,0.325522,0.000483
4,PUSCH,1,3,0.490234,2000,-2.595542,0.250040,0.000448
5,PUSCH,1,4,0.601562,24,-1.656561,0.869496,0.000500
6,PUSCH,1,4,0.601562,100,-1.876438,0.490493,0.000493
7,PUSCH,1,4,0.601562,500,-1.567656,0.059098,0.000413
8,PUSCH,1,4,0.601562,1000,-1.587657,0.045752,0.000048
9,PUSCH,1,4,0.601562,2000,-1.594308,0.040242,0.000248


Optionally, save the results to file

In [4]:
save_to_file = False
if save_to_file:
    df.to_csv('../data/bler_table_sigmoid_fit.csv', index=False)